In [1]:
import torch

In [2]:
def convert_files(sampling_file):
    with open(sampling_file, "r") as f:
        lines = f.readlines()

    X = [[int(l) for l in line[:-1]] for line in lines]

    return X

Data = [torch.tensor(convert_files("../sword_tail_fish/samples/ancestry-probs-par1_allchrs_TLMC_thinnedHMM.tsv_chrom_" + str(i))) - 1 for i in range(1,25)]


In [3]:
sample_width = 55
def get_sample(chrom1, start1, end1, chrom2, start2, end2):

    X1 = Data[chrom1 - 1]
    ind1 = int(1000*(start1 + end1)/2)

    sample1 = X1[:, ind1 - sample_width//2: ind1 + sample_width//2 + 1]

    X2 = Data[chrom2 - 1]
    ind2 = int(1000*(start2 + end2)/2)

    sample2 = X2[:, ind2 - sample_width//2: ind2 + sample_width//2 + 1]

    if not sample1.shape[1] == sample2.shape[1] == sample_width:
        return None
    
    sample = torch.cat((sample1,sample2),1)
    return sample

    

In [4]:
for i in range(24):
    print(Data[i].shape)

torch.Size([93, 571])
torch.Size([93, 481])
torch.Size([93, 500])
torch.Size([93, 599])
torch.Size([93, 692])
torch.Size([93, 519])
torch.Size([93, 543])
torch.Size([93, 485])
torch.Size([93, 484])
torch.Size([93, 638])
torch.Size([93, 502])
torch.Size([93, 483])
torch.Size([93, 648])
torch.Size([93, 693])
torch.Size([93, 412])
torch.Size([93, 431])
torch.Size([93, 739])
torch.Size([93, 1393])
torch.Size([93, 491])
torch.Size([93, 690])
torch.Size([93, 507])
torch.Size([93, 440])
torch.Size([93, 562])
torch.Size([93, 437])


In [5]:
with open("../sword_tail_fish/samples/Epistatic_pairs", "r") as f:
    lines = f.readlines()

lines = [line.split('\t') for line in lines]
lines = [line for line in lines if len(line) > 1]
lines.pop(0)

['chrom',
 'start_morgans',
 'end_morgans',
 'chrom',
 'start_morgans',
 'end_morgans\n']

In [6]:
lines = [[int(line[0]), float(line[1]), float(line[2]), int(line[3]), float(line[4]), float(line[5])] for line in lines]

In [7]:
threshold = 0.001
lines_certain = [line for line in lines if abs(line[1] - line[2]) < threshold and abs(line[4] - line[5]) < threshold]

In [8]:
len(lines)

327

In [9]:
len(lines_certain)

197

In [10]:
X = [get_sample(*line) for line in lines_certain]
len(X)

197

In [11]:
X = [x for x in X if x is not None]
len(X)

113

In [13]:
X = torch.stack(X)

In [14]:
X = torch.cat((X, X[:,:7,:]),1)

In [15]:
X = X.transpose(-2,-1)

In [16]:
X.shape

torch.Size([113, 110, 100])

In [55]:
from models import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = TransformerModel1()
model.load_state_dict(torch.load("full_model.pth", map_location=torch.device(device)))
model = model.to(device)
model.eval()

X = X.to(device)

y = model(X)
(y.sum().item(), (y > 0.5).sum().item(), y.shape)

cpu


(8.693506240844727, 1, torch.Size([113]))

In [56]:
import random
lines_certain2 = lines_certain.copy()
random.shuffle(lines_certain2)

Xfalse = [get_sample(*line1[:3], *line2[3:]) for line1, line2 in zip(lines_certain, lines_certain2)]
Xfalse = [x for x in Xfalse if x is not None]
Xfalse = torch.stack(Xfalse)
Xfalse = torch.cat((Xfalse, Xfalse[:,:7,:]),1)
Xfalse = Xfalse.transpose(-2,-1)
Xfalse = Xfalse.to(device)

y = model(Xfalse)
(y.sum().item(), (y > 0.5).sum().item(),  y.shape)

(10.013745307922363, 0, torch.Size([119]))